In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import shap
import lime
from scipy.stats import linregress
import lime.lime_tabular
import math
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import MinMaxScaler 
from keras.models import Sequential 
from keras.layers import Dense, LSTM 
from keras.layers import Dropout 
import matplotlib.pyplot as plt 
from tensorflow.keras.optimizers import Adam
from keras.layers import GRU 
from keras.layers import Flatten 
from keras.layers.convolutional import Conv1D, MaxPooling1D


2023-05-02 22:48:11.422725: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-02 22:48:11.564541: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/snzisad/anaconda3/lib/python3.9/site-packages/cv2/../../lib64:
2023-05-02 22:48:11.564561: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-05-02 22:48:12.205184: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dl

In [2]:
!pip install tensorflow
!pip install keras
!pip install shap
!pip install lime

In [3]:
def get_prediction_from_model(dataset):
    pred = selected_model.predict(dataset)

    return np.array(pred)

In [4]:

def get_lime_values(dataset):
    lime_values = []
    single_lime_value = []
    lime_explainer = lime.lime_tabular.RecurrentTabularExplainer(training_set, feature_names=features, class_names=['AQI'], verbose=True, mode='regression')

    for test_data in dataset:
      test_data = np.reshape(test_data, (1, test_data.shape[0],test_data.shape[1]))
      lime_exp = lime_explainer.explain_instance(test_data, get_prediction_from_model, num_features=len(features), labels=(1,))

      coeffs = lime_exp.local_exp[0]
      single_lime_value = [0] * len(features)
      for c in coeffs:
          single_lime_value[c[0]] += c[1]

      lime_values.append(single_lime_value)

    return np.array(lime_values)


In [5]:

def XAIEvaluation(model, model_name):
    global selected_model
    # Create a shap explainer
    selected_model = model
    # custom_training_set = np.reshape(training_set, (training_set.shape[0], training_set.shape[1], 1))
    
    shap_explainer = shap.DeepExplainer(model, training_set)
    
    X_test = dataset_x[num_trained_item:]
    y_test = dataset_y[num_trained_item:]
    
    
    
    num_items = 200
    subset_A = X_test[:num_items]
    subset_B = X_test[num_items:2*num_items]
    print(X_test.shape)
    # subset_A = np.reshape(subset_A, (subset_A.shape[0], subset_A.shape[1], 1))
    # subset_B = np.reshape(subset_B, (subset_B.shape[0], subset_B.shape[1], 1))
    
    shap_values_A = shap_explainer.shap_values(subset_A)
    shap_values_B = shap_explainer.shap_values(subset_B)
    
    shap_values = np.array([*shap_values_A, *shap_values_B])
    
    
    """### LIME Model"""
    
    
    num_items = 200
    subset_A = X_test[:num_items]
    subset_B = X_test[num_items:2*num_items]
    
    lime_values_A = get_lime_values(subset_A)
    lime_values_B = get_lime_values(subset_B)
    
    lime_values = np.array([*lime_values_A, *lime_values_B])
    
    # Calculate feature coverage
    shap_coverages = []
    
    nonzero_counts = np.sum(shap_values != 0, axis=0)
    nonzero_percents = nonzero_counts / shap_values.shape[0]
    shap_coverages.extend(nonzero_percents)
    
    
    
    # Calculate feature coverage
    lime_coverages = []
    
    nonzero_counts = np.sum(lime_values != 0, axis=0)
    nonzero_percents = nonzero_counts / lime_values.shape[0]
    lime_coverages.extend(nonzero_percents)
    
    
    
    # calculate the average absolute SHAP value for each feature
    avg_shap = np.abs(shap_values).mean(axis=0)
    
    
    """### LIME Model"""
    
    # calculate the average absolute lime value for each feature
    avg_lime = np.abs(lime_values).mean(axis=0)
    
    
    shap_consistency = np.mean(np.abs(shap_values_A.flatten() - shap_values_B.flatten()))
    
    
    slope, intercept, r_value, p_value, std_err = linregress(shap_values_A.flatten(), shap_values_B.flatten())
    shap_icc = (r_value**2) * (np.sum(shap_values_A.flatten()**2) / len(shap_values_A.flatten()))
    
    """### LIME Model"""
    
    lime_consistency = np.mean(np.abs(lime_values_A.flatten() - lime_values_B.flatten()))    
    
    slope, intercept, r_value, p_value, std_err = linregress(lime_values_A.flatten(), lime_values_B.flatten())
    lime_icc = (r_value**2) * (np.sum(lime_values_A.flatten()**2) / len(lime_values_B.flatten()))
    
    
    
    # Errors
    
    y_predict = model.predict(X_test)
    
    mae = mean_absolute_error(y_predict, y_test)
    mse = mean_squared_error(y_predict, y_test)
    rmse = math.sqrt(mse)
    
    
    row = {"Model Name": model_name, "MAE": mae, "MSE": mse, "RMSE": rmse, 
           "SHAP Coverage":shap_coverages, "SHAP Avg Coverage": np.mean(shap_coverages), "SHAP Relevence":avg_shap, "SHAP Avg Relevence":np.mean(avg_shap), "SHAP CE":shap_consistency, "SHAP ICC":shap_icc,
            "LIME Coverage":lime_coverages, "LIME Avg Coverage": np.mean(lime_coverages), "LIME Relevence":avg_lime, "LIME Avg Relevence":np.mean(avg_lime), "LIME CE":lime_consistency, "LIME ICC":lime_icc}
    output_file.append(row)
    pd.DataFrame.from_dict(output_file).to_csv(output_file_path, index=False)


In [6]:

def df_to_X_y(df, window_size):
#   df_as_np = df.to_numpy()
  X = []
  y = []
  for i in range(len(df)-window_size):
    row = [r for r in df[i:i+window_size]]
    X.append(row)
    label = df[i+window_size][-1]
    y.append(label)
    # print(label)
  return np.array(X), np.array(y)


In [7]:
root_path = '..'
updated_dataset = root_path+"/UpdatedAirQualityDataset.csv"
df = pd.read_csv(updated_dataset)

In [8]:
file_name = "ml_"+str(datetime.now())+".csv"
file_name = file_name.replace(' ', '-')
file_name = file_name.replace(':', '-')
output_file_path = root_path+"/result/"+file_name
output_file = []

In [9]:
features =list(df.columns[:-1])
num_trained_item = 10000

# # Min max scaler
scaler = MinMaxScaler()
scaler = scaler.fit(df)
df_for_training = scaler.transform(df)

WINDOW_SIZE = 10
X, y  = df_to_X_y(df_for_training, WINDOW_SIZE)
training_set = X[:500]
num_trained_item = 10000
dataset_x = X[:num_trained_item]
dataset_y = y[:num_trained_item]

X_train, X_test, y_train, y_test = train_test_split(dataset_x, dataset_y, test_size=0.2, random_state=42)

In [10]:

"""# SHAP Explanation"""

start = datetime.now()

num_epochs = 5

# LSTM
model_lstm = Sequential()
model_lstm.add(LSTM(units = 108, return_sequences = False, input_shape = (X_train.shape[1], X_train.shape[2]))) 
model_lstm.add(Dropout(0.2)) 
model_lstm.add(Dense(units = 1))

#compile and fit the model on 40 epochs 
optimizer = Adam(learning_rate=0.0001) 
model_lstm.compile(optimizer = optimizer, loss = 'mean_squared_error', metrics=['accuracy']) 
lstm_history = model_lstm.fit(X_train, y_train, epochs = num_epochs, batch_size = 32, shuffle=True, validation_split=0.20)

# # List all data in history 
# print(lstm_history.history.keys() )
# print("\n") 
# # summarize history for accuracy 
# plt.plot(lstm_history.history['loss']) 
# plt.plot(lstm_history.history['val_loss']) 
# plt.title('LSTM Learning Curve') 
# plt.ylabel('Loss') 
# plt.xlabel('Epoch') 
# plt. legend(['train', 'test'], loc='upper left') 
# plt.show()


2023-05-02 22:48:20.126246: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/snzisad/anaconda3/lib/python3.9/site-packages/cv2/../../lib64:
2023-05-02 22:48:20.126270: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-05-02 22:48:20.126284: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ZisadsLinux): /proc/driver/nvidia/version does not exist
2023-05-02 22:48:20.126457: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate c

Epoch 1/5
200/200 [==============================] - 4s 13ms/step - loss: 0.0164 - accuracy: 1.5625e-04 - val_loss: 0.0031 - val_accuracy: 0.0000e+00
Epoch 2/5
200/200 [==============================] - 2s 11ms/step - loss: 0.0031 - accuracy: 1.5625e-04 - val_loss: 0.0025 - val_accuracy: 0.0000e+00
Epoch 3/5
200/200 [==============================] - 2s 11ms/step - loss: 0.0029 - accuracy: 1.5625e-04 - val_loss: 0.0024 - val_accuracy: 0.0000e+00
Epoch 4/5
200/200 [==============================] - 2s 11ms/step - loss: 0.0027 - accuracy: 1.5625e-04 - val_loss: 0.0023 - val_accuracy: 0.0000e+00
Epoch 5/5
200/200 [==============================] - 2s 11ms/step - loss: 0.0026 - accuracy: 1.5625e-04 - val_loss: 0.0022 - val_accuracy: 0.0000e+00


In [11]:

selected_model = model_lstm
# custom_training_set = np.reshape(training_set, (training_set.shape[0], training_set.shape[1], 1))


X_test = X[num_trained_item:]
y_test = y[num_trained_item:]

num_items = 8
subset_A = X_test[:num_items]
subset_B = X_test[num_items:2*num_items]
print(subset_B.shape)


# shap_explainer = shap.DeepExplainer(model_lstm, training_set)
# # shap_explainer = shap.KernelExplainer(model_lstm.predict, training_set)

# shap_values_A = shap_explainer.shap_values(subset_A)
# shap_values_B = shap_explainer.shap_values(subset_B)

# shap_values = np.array([*shap_values_A, *shap_values_B])

(8, 10, 4)


In [22]:

j = 5
test_data = X_test[j:j+8]

explainer = shap.DeepExplainer(model_lstm, X_train)
# explainer = shap.KernelExplainer(model_lstm.predict, X_train)
shap_values = explainer.shap_values(test_data)
shap.initjs()
shap.force_plot(explainer.expected_value[0], shap_values[0][0][0], feature_names)


AttributeError: in user code:

    File "/home/snzisad/anaconda3/lib/python3.9/site-packages/shap/explainers/_deep/deep_tf.py", line 247, in grad_graph  *
        out = self.model(shap_rAnD)
    File "/home/snzisad/anaconda3/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "/home/snzisad/anaconda3/lib/python3.9/site-packages/shap/explainers/_deep/deep_tf.py", line 378, in custom_grad
        out = op_handlers[type_name](self, op, *grads) # we cut off the shap_ prefex before the lookup
    File "/home/snzisad/anaconda3/lib/python3.9/site-packages/shap/explainers/_deep/deep_tf.py", line 667, in handler
        return linearity_with_excluded_handler(input_inds, explainer, op, *grads)
    File "/home/snzisad/anaconda3/lib/python3.9/site-packages/shap/explainers/_deep/deep_tf.py", line 674, in linearity_with_excluded_handler
        assert not explainer._variable_inputs(op)[i], str(i) + 

In [19]:
j = 5
test_data = X_test[j]
test_data = np.reshape(test_data, (1, test_data.shape[0], test_data.shape[1]))
model_lstm.predict(test_data)[0][0]

1/1 [==============================] - 0s 28ms/step


0.3099918